In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
num_rows = 1000

In [3]:
# --- 1. Generate a clean base dataset ---
# Patient_ID: Benzersiz hasta id numarası
patient_ids = [i for i in range(1, num_rows + 1)]

# Age: Hastanın yaşı (Gerçekçi aralık)
ages = np.random.randint(1, 90, num_rows)

# Gender: Hastanın cinsiyeti
genders = random.choices(['Male', 'Female'], k=num_rows)

# Condition: Hastalık kondisyonu
conditions = random.choices([
    'Influenza', 'Hypertension', 'Diabetes', 'Asthma', 'Heart Disease',
    'Pneumonia', 'Appendicitis', 'Fracture', 'Migraine', 'Bronchitis'
], k=num_rows)

# Procedure: Uygulanan prosedürler
procedures = random.choices([
    'General Check-up', 'Surgery', 'Consultation', 'MRI Scan', 'Blood Test',
    'Physical Therapy', 'Chemotherapy', 'Vaccination', 'Dialysis', 'Endoscopy'
], k=num_rows)

# Cost: İşlemlerin maliyeti (Gerçekçi aralık)
costs = np.round(np.random.uniform(50, 10000, num_rows), 2)

# Length_of_stay: Yatış süresi (Gün)
lengths_of_stay = np.random.randint(1, 30, num_rows)

# Readmission: Yeniden kabul edilme, yatış (Evet/Hayır)
readmissions = random.choices(['No', 'Yes'], weights=[0.85, 0.15], k=num_rows)

# Outcome: Sonuç
outcomes = random.choices(['Discharged', 'Transferred', 'Deceased', 'Stable'], weights=[0.75, 0.10, 0.05, 0.10], k=num_rows)

# Satisfaction: Memnuniyet oranı
satisfactions = np.random.randint(1, 5, num_rows)

data = {
    'Patient_ID': patient_ids,
    'Age': ages,
    'Gender': genders,
    'Condition': conditions,
    'Procedure': procedures,
    'Cost': costs,
    'Length_of_stay': lengths_of_stay,
    'Readmission': readmissions,
    'Outcome': outcomes,
    'Satisfaction': satisfactions
}

df = pd.DataFrame(data)


In [4]:
df.head()

,Patient_ID,Age,Gender,Condition,Procedure,Cost,Length_of_stay,Readmission,Outcome,Satisfaction
0,1,37,Male,Hypertension,Dialysis,5803.06,5,No,Discharged,2
1,2,81,Male,Diabetes,Blood Test,9279.65,1,Yes,Discharged,1
2,3,47,Female,Fracture,Blood Test,653.88,19,No,Discharged,3
3,4,83,Female,Fracture,General Check-up,1690.47,19,No,Transferred,2
4,5,73,Female,Hypertension,Physical Therapy,9297.59,7,No,Discharged,1


In [5]:
# --- 2. Introduce Kirlilik (Orta Düzey) ---

In [6]:
# --- Eksik Değerler (Missing Values) ---
# %5-10 oranında eksik değer ekleyelim
for col in ['Condition', 'Procedure', 'Cost', 'Outcome', 'Satisfaction', 'Length_of_stay']:
    missing_indices = np.random.choice(df.index, size=int(num_rows * random.uniform(0.05, 0.10)), replace=False)
    df.loc[missing_indices, col] = np.nan

In [7]:
# --- Çift Kayıtlar (Duplicate Records) ---
# %2-3 oranında çift kayıt ekleyelim (bazılarını Patient_ID ile)
num_duplicates = int(num_rows * random.uniform(0.02, 0.03))
duplicate_rows = df.sample(n=num_duplicates, replace=True)
df = pd.concat([df, duplicate_rows], ignore_index=True)
# Bazı duplicate kayıtların Patient_ID'sini farklı yapalım ki tam duplicate olmasın ama bazı bilgileri aynı olsun
# Bu kısımda Patient_ID'nin sayısal olmasını sağlamak için int kullanacağız
for _ in range(int(num_duplicates * 0.3)):
    idx = random.choice(duplicate_rows.index)
    if idx < len(df): # Ensure index is valid for current df size
        # Yeni Patient_ID'nin mevcut ID'lerden farklı ve sayısal olmasını sağlamak
        # Mevcut en büyük ID'den sonraki sayılardan rastgele seçelim
        new_id = random.randint(df['Patient_ID'].max() + 1, df['Patient_ID'].max() + 1000)
        while new_id in df['Patient_ID'].values: # ensure uniqueness
            new_id = random.randint(df['Patient_ID'].max() + 1, df['Patient_ID'].max() + 1000)
        df.loc[idx, 'Patient_ID'] = new_id

In [8]:
# --- Veri Tipi Hataları (Data Type Inconsistencies) ---
# Age: Sayısal yerine metin, anlamsız karakterler
age_error_indices = np.random.choice(df.index, size=int(num_rows * 0.02), replace=False)
for idx in age_error_indices:
    if random.random() < 0.5:
        df.loc[idx, 'Age'] = random.choice(['yirmi', 'otuz beş', 'kırkaltı', 'NA', 'N/A', '##'])
    else:
        df.loc[idx, 'Age'] = np.random.randint(100, 150) # Outlier numeric

# Cost: Sayısal yerine metin, para birimi sembolleri
cost_error_indices = np.random.choice(df.index, size=int(num_rows * 0.03), replace=False)
for idx in cost_error_indices:
    if random.random() < 0.5:
        df.loc[idx, 'Cost'] = random.choice(['bir bin', 'iki bin beş yüz', 'bilinmiyor', 'FREE'])
    else:
        df.loc[idx, 'Cost'] = f"${np.round(np.random.uniform(10, 20000), 2)}" # Add currency symbol

# Length_of_stay: Sayısal yerine metin
los_error_indices = np.random.choice(df.index, size=int(num_rows * 0.02), replace=False)
for idx in los_error_indices:
    df.loc[idx, 'Length_of_stay'] = random.choice(['bir gün', 'iki hafta', 'uzun süre', 'NaN'])

# Satisfaction: Sayısal olmasına rağmen farklı formatlar (metin, anlamsız sayılar)
satisfaction_error_indices = np.random.choice(df.index, size=int(num_rows * 0.05), replace=False)
for idx in satisfaction_error_indices:
    df.loc[idx, 'Satisfaction'] = random.choice(['High', 'Low', 'Not Rated', '5', '0', '-1']) # 1-4 dışı veya metin


/var/folders/rq/whc6dkh12hlgmtffmj488qn80000gn/T/ipykernel_47784/1034225162.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NA' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[idx, 'Age'] = random.choice(['yirmi', 'otuz beş', 'kırkaltı', 'NA', 'N/A', '##'])
/var/folders/rq/whc6dkh12hlgmtffmj488qn80000gn/T/ipykernel_47784/1034225162.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '$3878.01' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[idx, 'Cost'] = f"${np.round(np.random.uniform(10, 20000), 2)}" # Add currency symbol
/var/folders/rq/whc6dkh12hlgmtffmj488qn80000gn/T/ipykernel_47784/1034225162.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of panda

In [9]:
# --- Yazım Yanlışları ve Tutarsızlıklar (Typos and Inconsistencies) ---
# Gender
gender_map = {
    'Male': ['male', 'MALE', 'm', 'Erkek', 'male '],
    'Female': ['female', 'FEMALE', 'f', 'Kadın', 'female ']
}
gender_typo_indices = np.random.choice(df.index, size=int(num_rows * 0.05), replace=False)
for idx in gender_typo_indices:
    current_gender = df.loc[idx, 'Gender']
    if pd.notna(current_gender) and current_gender in gender_map:
        df.loc[idx, 'Gender'] = random.choice(gender_map[current_gender])
    elif pd.notna(current_gender): # Introduce new inconsistent values if gender is not in original map
        df.loc[idx, 'Gender'] = random.choice(['Unspecified', 'Other'])

# Condition, Procedure, Outcome: Yazım hataları, ekstra boşluklar, büyük/küçük harf
def introduce_typo(text):
    if pd.isna(text):
        return text
    text = str(text)
    if random.random() < 0.3: # Change casing
        text = text.lower() if random.random() < 0.5 else text.upper()
    if random.random() < 0.2: # Add extra spaces
        text = text + ' ' * random.randint(1, 3)
    if random.random() < 0.1: # Introduce a simple typo
        if len(text) > 3:
            idx = random.randint(0, len(text) - 1)
            char_list = list(text)
            char_list[idx] = random.choice('abcdefghijklmnopqrstuvwxyz')
            text = ''.join(char_list)
    return text

for col in ['Condition', 'Procedure', 'Outcome']:
    df[col] = df[col].apply(introduce_typo)

In [10]:
# --- Aykırı Değerler (Outliers) ---
# Age: Çok düşük veya çok yüksek
age_outlier_indices = np.random.choice(df.index, size=int(num_rows * 0.01), replace=False)
for idx in age_outlier_indices:
    df.loc[idx, 'Age'] = random.choice([0, 1, 105, 110, 120])

# Cost: Aşırı düşük veya aşırı yüksek
cost_outlier_indices = np.random.choice(df.index, size=int(num_rows * 0.01), replace=False)
for idx in cost_outlier_indices:
    df.loc[idx, 'Cost'] = random.choice([1, 10, 50000, 100000])

# Length_of_stay: Çok kısa veya çok uzun
los_outlier_indices = np.random.choice(df.index, size=int(num_rows * 0.01), replace=False)
for idx in los_outlier_indices:
    df.loc[idx, 'Length_of_stay'] = random.choice([0, 1, 60, 90, 100])

# Satisfaction: Aykırı sayısal değerler (1-4 dışı)
satisfaction_outlier_indices = np.random.choice(df.index, size=int(num_rows * 0.01), replace=False)
for idx in satisfaction_outlier_indices:
    df.loc[idx, 'Satisfaction'] = random.choice([0, 5, 6, 10]) # 1-4 dışı sayılar


In [11]:
# Karışık ve final hale getir (indeksleri sıfırla ve karıştır)
df = df.sample(frac=1).reset_index(drop=True)

In [12]:
# Print the first few rows and info to show the "dirtiness"
print("Oluşturulan 'kirli' veri setinin ilk 5 satırı:")
print(df.head())
print("\nVeri seti hakkında bilgi:")
print(df.info())
print("\nEksik değerlerin özeti:")
print(df.isnull().sum())

Oluşturulan 'kirli' veri setinin ilk 5 satırı:
   Patient_ID Age  Gender         Condition         Procedure     Cost  \
0         606  40    Male               NaN  General Check-up  2728.06   
1         671  12  Female          Diabetes          MRI Scan    85.16   
2         327  50  Female  HEART tISEASE          MRI Scan  n      NaN   
3         419  67    Male          Diabetes          Dialysis  8469.21   
4         409   4    Male        sronchitis      Consultation   810.31   

  Length_of_stay Readmission       Outcome Satisfaction  
0           12.0          No  Discharged            3.0  
1           25.0         Yes    Discharged          4.0  
2           15.0          No           NaN          2.0  
3            8.0          No    DISCHARGED          4.0  
4            NaN          No    Discharged          2.0  

Veri seti hakkında bilgi:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1022 entries, 0 to 1021
Data columns (total 10 columns):
 #   Column          Non-N

In [13]:
df.head()

,Patient_ID,Age,Gender,Condition,Procedure,Cost,Length_of_stay,Readmission,Outcome,Satisfaction
0,606,40,Male,NaN,General Check-up,2728.06,12.0,No,Discharged,3.0
1,671,12,Female,Diabetes,MRI Scan,85.16,25.0,Yes,Discharged,4.0
2,327,50,Female,HEART tISEASE,MRI Scan n,NaN,15.0,No,NaN,2.0
3,419,67,Male,Diabetes,Dialysis,8469.21,8.0,No,DISCHARGED,4.0
4,409,4,Male,sronchitis,Consultation,810.31,NaN,No,Discharged,2.0


In [14]:
df.to_csv('hospital_dataset.csv', index=False, encoding='utf-8')